# Gaussian Mixture

One can see this unsipervised learning model as a "soft" version of K-Means. In K-Means, for each cluster, the datapoints were separated between belongs to or does not belong to the cluster. Now, what we want is a probability to belong to the cluster. In other words, given a datapoint we would like to compute its probability of belonging to each cluster. For instance, if we look for two clusters, one datapoint will have probability 0.2 to belong to cluster 1 and probability 0.8 of belonging to cluster 2.

As in K-Means, one needs to provide the number of cluster looked for. There exists some extension that are able in certain cases to detect whether the number of cluster given is too low or too high but things will be kept simple for now.

In order to do find the different probability, it seems natural to try to modelise the clusters by probability distributions. For Gaussian Mixture, the clusters are modelised, as the name indicates, by multivariate gaussians. Each cluster is thus supposed to follow a probability distribution of $N(\mu, \Sigma)$ and the whole idea is to find those $\mu$ and $\Sigma$.